In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

from webdriver_manager.chrome import ChromeDriverManager

import requests
import csv
import os

import numpy as np
import pandas as pd
from numpy import random
from time import sleep

import BATParser

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
# driver = webdriver.Chrome()
driver.get('https://bringatrailer.com/toyota/land-cruiser-fj60-fj62/')

In [ ]:
# extend page
while True:
    try:
        button = driver.find_element(By.XPATH , "//div[@class='auctions-footer auctions-footer-previous']/*")
        button.click()
    except NoSuchElementException:
        break
    sleeptime = random.uniform(3, 5)
    sleep(sleeptime)

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

In [ ]:
listings = soup.find('div', {'class': 'blocks'}).find_all('div', {'class': 'block'})
links = []
for listing in listings:
    link = listing.a.get('href')
    links.append(link)
# remove below link because it was REMOVED from auction, but page still exists (i.e., auction did not finish)
links.remove('https://bringatrailer.com/listing/1984-toyota-land-cruiser-fj60-7/')

In [ ]:
parser = BATParser.Parser()
with open ('landcruisers.csv', 'w', newline='', encoding='utf-8') as csv_file:
    # format csv
    csv_writer = csv.writer(csv_file)
    # header
    csv_writer.writerow(['title','price','sell_date', 'no_views','no_watchers','no_comments', 'no_bids', 'seller_state', 
                         'year', 'series', 'miles','engine','trans','exterior','interior', 'misc', 'url'])
    for url in links:
        source = requests.get(url).text
        
        scraped_page = parser.parse(source)
        scraped_page.append(url)
        csv_writer.writerow(scraped_page)

        sleeptime = random.uniform(20, 60)
        sleep(sleeptime)

In [ ]:
# https://bringatrailer.com/listing/1985-toyota-land-cruiser-fj60-2/ 

In [ ]:
# https://bringatrailer.com/listing/1984-toyota-land-cruiser-fj60-7/
# causes machine to break

In [3]:
import BATParser
parser = BATParser.Parser()

In [8]:
url = 'https://bringatrailer.com/listing/1989-toyota-land-cruiser-3/'
source = requests.get(url).text
parser.parse(source)

['1989 Toyota Land Cruiser HJ61',
 '10,000',
 '8/15/17',
 '22,057',
 '314',
 '44',
 '27',
 'rego park, new york 11374',
 '1989',
 'hj61',
 '144,881',
 '4.0 liter 12h-t inline-six turbodiesel engine',
 'automatic',
 nan,
 nan,
 ['chassis: hj61014296',
  'japanese domestic model rhd',
  '233,163 kms (144,881 miles), 134,332 kms shown',
  '4.0 liter 12h-t inline-six turbodiesel engine',
  'automatic transmission',
  'new 4wd solenoid',
  'new front rotors and pads',
  '1.5"/2" lift',
  'old man emu steering stabilizer',
  'r134a conversion']]